# Training the tensorflow model on Google Cloud AI Platform

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/Breast-Cancer-Detection

In [ ]:
# without this, cv2 throws an error 
!sudo apt-get install ffmpeg libsm6 libxext6  -y

In [3]:
import os

In [5]:
PROJECT = 'vernal-buffer-285906'
REGION = 'us-west1'
BUCKET = 'vernal-buffer-285906'

In [6]:
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.3'

In [7]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [8]:
# os.environ['TRAIN_DIR'] = "/home/jupyter/Breast-Cancer-Detection/data"
# os.environ['OUTPUT_DIR'] = "/home/jupyter/Breast-Cancer-Detection/model_results"

os.environ['TRAIN_DIR'] = 'test_cnn'
os.environ['OUTPUT_DIR'] = 'gs://vernal-buffer-285906/model_results'

In [30]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/tf_trainer
#Train using python
time python -m trainer.task \
 --input-dir=$TRAIN_DIR \
 --output-dir=$OUTPUT_DIR \
 --bucket=$BUCKET \
 --num-epochs=75 \
 --batch_size=10 \
 --steps-per-epoch=5

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 256)       2

2021-03-11 14:44:26.966379: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2200200000 Hz
2021-03-11 14:44:26.967186: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561b20e54500 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-03-11 14:44:26.967223: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-03-11 14:44:26.967379: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Copying gs://vernal-buffer-285906/test_cnn/all_images.zip...
/ [1 files][ 10.6 MiB/ 10.6 MiB]                                                
Operation completed over 1 objects/10.6 MiB.                                     
2021-03-11 14:44:31.378916: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2021-03-11 14:44:35.67440

In [24]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/tf_trainer
    
OUTDIR=$OUTPUT_DIR
JOBNAME=breastcancer_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gcloud ai-platform jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/tf_trainer/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=BASIC \
  --runtime-version=2.3 \
  --python-version=3.7 \
  -- \
  --bucket=$BUCKET \
  --output-dir=$OUTDIR \
  --input-dir=$TRAIN_DIR \
  --num-epochs=75 \
  --batch_size=10 \
  --steps-per-epoch=5

gs://vernal-buffer-285906/model_results us-west1 breastcancer_210311_122832
jobId: breastcancer_210311_122832
state: QUEUED


Job [breastcancer_210311_122832] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe breastcancer_210311_122832

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs breastcancer_210311_122832


In [ ]:
# Running TensorBoard
# Open the port you are supposed to use (usually 6006) in Google cloud vm instance
# info -> https://docs.bitnami.com/google/faq/administration/use-firewall/
# Run tensorboard using following command
# tensorboard --logdir <path/to/log/directory> --bind_all
# Ex:
# tensorboard --logdir gs://vernal-buffer-285906/model_results/tensorboard --bind_all